# Codes for my Bachelorthesis

## Read the data from the provided blv files

In [1]:
# crucial imports to make the codes work
# the blv analysis works with Magpy, an open source programm to deal with geomagnetic data

import sys
sys.path.insert(1,'/path/to/magpy/') # should be path to magpy2
from magpy.stream import *
from magpy.core.methods import *
from magpy.core import plot as mp
from magpy.core import flagging
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
%matplotlib inline
import os
import pandas as pd

In [5]:
# identify all observatory codes with blv files

destination = '/path/to/blvdata/' #change path to folder with all blv files

obscodelist = []

# Walk through the directory tree, searching for files
for root, dirs, files in os.walk(destination):
    for file in files:
        # Check if the file has the ".blv" extension
        if file.endswith(".blv"):
             # Extract the first 3 characters of the filename (assumed to be the observatory code)
             obscodelist.append(file[:3])
obscodelist = sorted(list(set(obscodelist)))
print (obscodelist)

['aaa', 'aae', 'abg', 'abk', 'aia', 'ale', 'ams', 'api', 'aqu', 'ars', 'asc', 'asp', 'bdv', 'bel', 'bfe', 'bfo', 'blc', 'bmt', 'bng', 'bou', 'box', 'brd', 'brw', 'bsl', 'cbb', 'cki', 'clf', 'cmo', 'cnb', 'cpl', 'csy', 'cta', 'cyg', 'czt', 'ded', 'dlr', 'dlt', 'dmc', 'dou', 'drv', 'dur', 'ebr', 'esk', 'eyr', 'fcc', 'frd', 'frn', 'fur', 'gan', 'gck', 'gdh', 'gln', 'gna', 'gng', 'gua', 'gui', 'gzh', 'had', 'hbk', 'her', 'hlp', 'hon', 'hrb', 'hrn', 'hua', 'hyb', 'ipm', 'iqa', 'irt', 'izn', 'jai', 'jco', 'kak', 'kdu', 'kep', 'khb', 'kiv', 'kmh', 'kny', 'kou', 'ler', 'lnp', 'lon', 'lov', 'lrm', 'lvv', 'lyc', 'lzh', 'mab', 'maw', 'mbc', 'mbo', 'mcq', 'mea', 'mgd', 'mid', 'mmb', 'naq', 'nck', 'new', 'ngk', 'nur', 'nvs', 'orc', 'ott', 'paf', 'pag', 'pbq', 'peg', 'pet', 'phu', 'pil', 'ppt', 'pst', 'qsb', 'res', 'sba', 'sbl', 'sfs', 'she', 'shu', 'sit', 'sjg', 'sod', 'spg', 'spt', 'stj', 'stt', 'sua', 'tam', 'tan', 'tdc', 'teo', 'thl', 'thy', 'trw', 'tsu', 'ttb', 'tuc', 'ups', 'val', 'vic', 'vna'

## Plot the observed basevalues and the adopted baseline

In [ ]:
# Plot a single observatory (i.e. ott)

data = read(os.path.join(destination, 'ott*'), mode= None)
dataad = read(os.path.join(destination, 'ott*'), mode= 'adopted')
print(len(data), data.header.get("DataComponents"))
mp.tsplot([data, dataad],     
          [['dx','dy','dz']], 
          symbols=[['.', '.', '.'],['-','-','-']], 
          title=data.header.get('StationID'), 
          padding=[[2,0.005,2]], 
          symbolcolor=['grey','red'], 
          height=2)
# Optionally save the plot as an image (commented out here)
#plt.savefig("/path/to/plot_folder/ott.png")

In [ ]:
# Plot the baselines for all observatory in obscodelist with for-loop

for el in obscodelist:
    data = read(os.path.join(destination, '{}*'.format(el)), mode=mode)
    dataad = read(os.path.join(destination, '{}*'.format(el)), mode= 'adopted')
    print(len(data), data.header.get("DataComponents"))
    mp.tsplot([data, dataad],[['dx','dy','dz']], symbols=[['.', '.', '.'],['-','-','-']], title=data.header.get('StationID'), padding=[[2,0.005,2]], symbolcolor=['grey','red'], height=2)
    # Optionally save the plot as an image (commented out here)
    #plt.savefig("/path/to/plot_folder/mixed_baselines/{}.png".format(el))

## Adjust adopted data

- correct jumps in data with adding the difference of the values before and after a jump to every value after the jump
- add a fitting function (polynomial with degree 2) to cancel out bigger variations, linear increase, etc. 

#### Define functions

In [ ]:
# Jump and drift correction in a single method

def correct_jumps(data, jumps=True, drift=True, debug=False):
    """
    Identifying and correcting jumps
    """

    # Identifying and correcting jumps  
    # ----------------------------------------
    # Step 1
    # Using the first derivative to find steep changes in the data set asociated with jumps.
    # Indices are determined for all three baseline components. Jumps are identified by exceeding
    # a certain threshold of the overall standard deviation of the derivative

    def drop_sub(l, diff=1):
        # remove subsequent indices which differ equal less than diff
        index = 1
        while index < len(l):
            if l[index] - l[index - 1] <= diff:
                del l[index]
            else:
                index += 1
        return l

    testdata = data.copy()
    testdata = testdata.derivative(keys=['dx','dy','dz'],put2keys=['x','y','z'])

    inds = []
    sensitivity = 1
    for idx,comp in enumerate(['x','y','z']):
        sensitivity = 1
        mx,mdx  = testdata.mean(comp,std=True)
        # get indicies of all values exceeding stddev
        col = testdata._get_column(comp)
        tinds = np.argwhere(np.abs(col) > sensitivity*mdx)
        # assume that not more then 10 jumps are present in the timeseries and gradually increase the threshold until less then 10 jumps are found 
        while len(tinds) > 40:
            sensitivity += 1
            tinds = np.argwhere(np.abs(col) > sensitivity*mdx)
        # add start and end index for the timeranges
        nind = [i[0] for i in tinds]
        nindstart = [0]
        nindend = len(col)-1
        nindstart.extend(nind)
        nindstart.append(nindend)
        # remove subsequent (diff 1) indices
        nindstart = drop_sub(nindstart, diff=1)
        inds.append(nindstart)
    if debug:
        print ("Indices with jumps indications", inds)

    # Step 2
    # Combining the jumps to a single list, asuming that jumps affect all components (this is not necessarily true: i.e.
    # horizontal rotation of the sensor will only affect x,y but not z)
    tempinds = inds[0] + list(set(inds[1]) - set(inds[0]))
    totalinds = sorted(tempinds + list(set(inds[2]) - set(tempinds)))
    totalinds = drop_sub(totalinds, diff=2)
    print (totalinds)

    # Create range data [[start1,end1],[start2,end2],...]
    end = 0
    ar = []
    for idx,el in enumerate(totalinds):
        if idx > 0:
            if end:
                start = end + 2
            else:
                start = 0
            end = el
            ar.append([start,end])
    if debug:
        print ("Combined indices to ranges", ar)

    # Step 3
    # Extract specifoc timeranges from data, obtain offsets relative to previous data set and correct
    # 
    tcol = testdata._get_column('time')
    newdata = DataStream()
    lastx = 0
    lasty = 0
    lastz = 0
    for el in ar:
        # define each segment
        starttime = tcol[el[0]]
        endtime = tcol[el[1]]
        tdata = testdata.copy()
        tdata = tdata.trim(starttime,endtime)
        firstx = tdata._get_column('dx')[0]
        firsty = tdata._get_column('dy')[0]
        firstz = tdata._get_column('dz')[0]
        offx = lastx-firstx
        offy = lasty-firsty
        offz = lastz-firstz
        if debug:
            print ("Absolute offsets at {}: {}={:.2f}{}, {}={:.2f}{}, {}={:.2f}{}".format(endtime, tdata.header.get('col-dx'), offx, tdata.header.get('unit-col-dx'), tdata.header.get('col-dy'),offy, tdata.header.get('unit-col-dy'), tdata.header.get('col-dz'),offz, tdata.header.get('unit-col-dz')))
        # normailze by means
        tdata = tdata.offset({'dx': offx,'dy': offy,'dz': offz})
        lastx = tdata._get_column('dx')[-1]
        lasty = tdata._get_column('dy')[-1]
        lastz = tdata._get_column('dz')[-1]
        newdata = join_streams(tdata,newdata)
    #mp.tsplot(newdata, keys=['dx','dy','dz'])    

    # Step 4
    if drift:
        # Removing non-stationary contribution by fitting a polynomial function
        func = newdata.fit(keys=['dx','dy','dz'], fitfunc='poly',degree=2)
        #mp.tsplot(newdata, keys=['dx','dy','dz'], functions=[[func,func,func]], height=2)
        # and the subtracting these function from the data set
        corrdata = newdata.func2stream(func, keys=['dx','dy','dz'],mode='sub')
    else:
        corrdata = newdata

    return corrdata

In [ ]:
# Application
# ---------------------
obscode='wic'
data = read(os.path.join(destination, "{}*".format(obscode)), mode="adopted")
corrdata = correct_jumps(data, drift=True, debug=True)
mp.tsplot(corrdata, keys=['dx','dy','dz'], height=2)

#### Use defined functions

## Find periodicities with a power spectral analysis and Amplitude, DOY Max and Min with the average yearly baseline

#### Define function

In [ ]:
# powerspectrum
def powerspectrum(trimdata, component='dz', number=0):
    # Extract the time column from the provided data
    T = trimdata._get_column('time')
    print (len(T))
    # Create an array of time points, linearly spaced
    t = np.linspace(0,len(T),len(T))
    # Extract the signal data (default component is 'dz')
    h = trimdata._get_column(component)
    # Get the sampling rate
    sr = trimdata.samplingrate() # in seconds
    fs = 1./sr # Calculate the frequency sampling rate (Hz)
    # Plot the signal over time in the first subplot
    fig, (ax0, ax1) = plt.subplots(2, 1, layout='constrained')
    ax0.plot(t, h)
    ax0.set_xlabel('Time')
    ax0.set_ylabel('Signal')
    # Calculate the power spectrum using the periodogram (psd function)
    power, freqs = psd(h, NFFT=len(t), pad_to=len(t), Fs=fs, detrend='mean', scale_by_freq=True)
    plt.xscale('log') # logarithmic scale
    plt.show()
    # Import functions to find peaks in the power spectrum
    from scipy.signal import find_peaks, peak_prominences
    # Plot the power spectrum with logarithmic scale for both axes
    fig = plt.figure()
    plt.plot(freqs, power)
    plt.yscale('log')
    plt.xscale('log')
    # Find prominent peaks in the power spectrum (within the first 500 frequencies)
    minprom=np.max(power)-np.max(power)*0.90 # Set minimum prominence threshold
    peaks, _=find_peaks(power[:500], prominence=(minprom,None))
    # Calculate the period of the identified peaks (in days)
    period=1/freqs[peaks]/86400
    print(period)
    plt.title("Powerspectrum for {} with periodicities: {}".format(code[number], period))
    plt.plot(freqs[peaks], power[peaks], 'x')
    # Optionally save the plot as an image (commented out here)
    #plt.savefig("/path/to/plot_folder/power/{}.png".format(code[number]))
    plt.show

In [ ]:
# Function to calculate and display the average yearly baseline for a given component
def show_yearly_average(data, component='dy', show=True, number=0, code=None):
    # Define the range of years (from 1990 to 2023)
    years = np.arange(1990,2024,1)
    full = []
    # Loop over each year in the specified range
    for year in years:
        st = "{}-01-01".format(year)
        et = "{}-01-01".format(year+1)
        # Trim the data to the current year (extract the data for this year)
        trimdata = data.trim(st,et)
        # Get the specified column (default 'dy') from the trimmed data
        valcol = trimdata._get_column(component)
        doy = np.arange(1,366,1)
        # Check if the length of the column matches the expected number of days (365 or 366)
        if len(valcol) in [365,366]:
            if len(valcol) == 366:
                # Drop 29 February
                valcol = np.asarray(list(valcol[:59])+ list(valcol[60:]))
             # Append the adjusted values (mean-centered) for this year to the list
            full.append(valcol-np.mean(valcol))
    # Calculate the average and standard deviation across all years (ignoring NaNs)
    m = np.nanmean(full, axis=0)
    s = np.nanstd(full, axis=0)
    # Find the day of the year with the maximum and minimum average values
    doymax = np.argmax(m)
    doymin = np.argmin(m)
    # Get the corresponding values for the max and min days
    maxval = np.max(m)
    minval = np.min(m)
    # Check if the amplitude (difference between max and min) is valid
    amplitude_valid = m[doymax] - s[doymax] > m[doymin] + s[doymin]

    # If amplitude is valid, print the results
    if amplitude_valid:
        print(f"Max: Day {doymax}, Value {maxval}")
        print(f"Min: Day {doymin}, Value {minval}")
    else:
        print("No clear amplitude detected.")
        maxval = "none"
        minval = "none"
    
    # Save the results (max and min days and values) to a CSV file
    results = {
        'Day of Max': [doymax],
        'Day of Min': [doymin],
        'Amplitude Max': [maxval],
        'Amplitude Min': [minval]
    }
    #save_results_to_csv(f"/path/to/csv_folder/to/save/csv_file/{code}_yearly_average.csv", results)

    # Optionally, plot and show/save the results
    if show:
        fig = plt.figure()
        plt.title("Average yearly baseline for {}".format(code))
        plt.plot(doy,m, color='black')
        plt.fill_between(doy, m+s, m-s, color='blue', alpha=0.2)
        # Optionally save the plot as an image (commented out here)
        #plt.savefig("/path/to/plot_folder/maxmin/{}.png".format(code), dpi=300)
        plt.close(fig)

#### Use functions for all obervatories in obscodelist

In [ ]:
# Application
# ---------------------
keys=['dx','dy','dz']

for el in obscodelist:
    data = read(os.path.join(destination, "{}*".format(el)), mode="adopted")
    mp.tsplot(data, keys=['dx','dy','dz'], height=2, symbolcolor=['red'])
    plt.close()
    data = data.interpolate_nans(['dx','dy','dz'])
    corrdata = correct_jumps(data, drift=True, debug=False)
    mp.tsplot([corrdata], keys=[['dx','dy','dz']], height=2, symbolcolor=['green'])
    #plt.savefig('/path/to/plot_folder/corrected/{}.png'.format(el))
    plt.close()
    mp.tsplot([data, corrdata], keys=[['dx','dy','dz']], height=2, symbolcolor=['red','green'])
    #plt.savefig("/path/to/plot_folder/comparsion/{}.png".format(el))
    plt.close()
    for key in keys:
        analysisdata = corrdata.copy()
        powerspectrum(analysisdata, component=key, code=obscode)
        show_yearly_average(analysisdata, component=key, show=True, code=obscode)

## Work with analyzed data

### Plot data on a world map

#### Import world map

In [ ]:
#crucial imports to plot world maps

import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import geodatasets
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
from PIL import Image 

download geodatasets/ne_110m_land.zip to import worldmap

#### Determine longitude and latitude from the observatories

In [ ]:
# read CSV-files
df1 = pd.read_csv("/path/to/analyzed/data.csv", delimiter=";") # Read the CSV file with the analysed data
df2 = pd.read_csv("/path/to/iagaList.csv", delimiter=";") # Read the CSV file with observatory information, such as latitude and longitude

# Convert the codes in both DataFrames to lowercase to ensure consistent comparison
df1['code'] = df1['code'].str.lower()
df2['code'] = df2['code'].str.lower()

# Merge the two DataFrames based on the 'code' column (matching codes from both files)
result = pd.merge(df1, df2, on='code')

# Doppelte Einträge in der ersten Spalte entfernen# Remove duplicate entries in the merged DataFrame based on the 'code' column
result = result.drop_duplicates(subset=['code'])

# Ergebnis anzeigen
#print(result)

# Save the merged and cleaned DataFrame to a new CSV file (including both analyzed data and observatory coordinates)
result.to_csv('/path/to/output.csv', index=False)

#### Plot datasets (CSV files) on the world map

In [ ]:
# An easy example, how to plot world maps with data. All of my maps are plotted like this, some with more data than other

# Read data from CSV files containing the necessary information
df = pd.read_csv("/path/to/data/i.e./with/periodicity.csv", delimiter=',', skiprows=0, low_memory=False)
df1 = pd.read_csv("/path/to/data/i.e./without/periodicity.csv", delimiter=',', skiprows=0, low_memory=False)

# Load the world map shapefile for plotting
worldmap = gpd.read_file('path/to/geodataset/ne_110m_admin_0_countries.zip')
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

# Plot the basic map of the world with countries outlined
worldmap.plot(
    ax=ax,
    color="lightgray",
    edgecolor="black",
    alpha=0.5  # Set transparency for better overlay visibility
)

# Extract longitude and latitude values for the first dataset
x = df['Longitude']
y = df['Latitude']
# Plot the data points for the first dataset (i.e. annual periodicity) in red
plt.scatter(x, y,
              alpha=0.6,             
              c='red',
              label='annual periodicity'
            )

# Extract longitude and latitude values for the second dataset
a = df1['Longitude']
b = df1['Latitude']
# Plot the data points for the second dataset (i.e. no annual periodicity) in blue
plt.scatter(a, b,
              alpha=0.6,             
              c='blue',
              label='no annual periodicity'
           )

# Annotate each data point from the datasets with the corresponding 'code'
for i, txt in df.iterrows():
    plt.annotate(txt['code'], (x[i], y[i]), fontsize=8, color='red')

for i, txt in df1.iterrows():
    plt.annotate(txt['code'], (a[i], b[i]), fontsize=8, color='blue')
    

# Creating axis limits and title
plt.xlim([-180, 180])
plt.ylim([-90, 90])

plt.legend(loc='lower right')
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# Optionally, save the plot to a file (this line is commented out)
#plt.savefig('path/to/worldmap/output.png', dpi=fig.dpi)
plt.show()

#### Determine the european observatories
(There are many European observatories, and since they are not easily visible on the world map, an additional Europe map will be plotted.)

In [ ]:
# Read data from a CSV file containing the necessary information (assuming it has 'Longitude' and 'Latitude' columns)
# The data is stored in 'af', with the file located at the specified path
af = pd.read_csv("/path/to/analyzed/data.csv", delimiter=',', skiprows=0, low_memory=False)

# Create a new geometry column by combining the 'Longitude' and 'Latitude' columns into Point objects
# This step creates a geometric representation of each location
geometry = [Point(xy) for xy in zip(af['Longitude'], af['Latitude'])]

# Create a GeoDataFrame (gdf) using the original data ('af') and the new 'geometry' column
# A GeoDataFrame is a special kind of DataFrame that handles geometric data
gdf = gpd.GeoDataFrame(af, geometry=geometry)

# Define the bounding box coordinates that cover the region of Europe
# (Longitude between -30 and 50, Latitude between 30 and 75)
min_lon, min_lat = -30, 30
max_lon, max_lat = 50, 75

# Filter the rows of the GeoDataFrame (gdf) that lie within the specified bounding box for Europe
# The 'cx' indexer is used to select data within the defined longitude and latitude ranges
europe_gdf = gdf.cx[min_lon:max_lon, min_lat:max_lat]

# Drop the 'geometry' column from the filtered GeoDataFrame as it's no longer needed for the output
# Save the remaining data (excluding the 'geometry' column) to a new CSV file
europe_gdf.drop(columns='geometry').to_csv("/path/to/EU/output.csv", index=False)

#### Plot European Observatories on an European Map

In [ ]:
import cartopy.crs as ccrs # Import Cartopy's coordinate reference system (CRS) tools
import cartopy.feature as cfeature # Import Cartopy's feature set, which includes borders and coastlines

# Reading two CSV files containing data for plotting (including 'Longitude' and 'Latitude')
cef = pd.read_csv("/path/to/EU/output1.csv", delimiter=',', skiprows=0, low_memory=False)
cef1 = pd.read_csv("/path/to/EU/output2.csv", delimiter=',', skiprows=0, low_memory=False)

# Define the geographical extent (bounding box) for the map to cover Europe
# The extent specifies the limits of the plot in [min_lon, max_lon, min_lat, max_lat]
extent = [-20, 40, 33, 70]

fig = plt.figure(figsize=(10, 10))
# Add a subplot to the figure with a PlateCarree projection (a simple latitude-longitude grid)
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
# Set the map extent based on the defined limits (Europe region)
ax.set_extent(extent)

# Plot data from the first CSV file (cef) on the map as red points
# Extract 'Longitude' and 'Latitude' from the DataFrame to create scatter plot
x = cef['Longitude']
y = cef['Latitude']
plt.scatter(x, y,
              alpha=0.6,             
              c='red',
              label='annual periodicity'
            )

# Plot data from the second CSV file (cef1) on the map as blue points
# Extract 'Longitude' and 'Latitude' from the second DataFrame to create scatter plot
a = cef1['Longitude']
b = cef1['Latitude']
plt.scatter(a, b,
              alpha=0.6,             
              c='blue',
              label='no annual periodicities'
           )

# Annotate each point from the DataFrames with its corresponding 'code'
for i, txt in cef.iterrows():
    plt.annotate(txt['code'], (x[i], y[i]), fontsize=10, color='red')

for i, txt in cef1.iterrows():
    plt.annotate(txt['code'], (a[i], b[i]), fontsize=10, color='blue')


# Borders of countries are added with dashed lines and 1px width
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=1)
# Coastlines are added to show the outline of landmasses
ax.add_feature(cfeature.COASTLINE)

ax.set_xlabel('Longitude')  # x-Achsenbeschriftung
ax.set_ylabel('Latitude')   # y-Achsenbeschriftung

ax.set_xticks(range(-20, 41, 10))  # Setze x-Achsen-Ticks von -20 bis 40
ax.set_yticks(range(33, 76, 10))

plt.legend(loc='upper left')

# Optionally, save the map to a PNG file (this line is commented out, but can be used if needed)
#plt.savefig('/path/to/European/map.png', dpi=fig.dpi)

plt.show()